In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load tokenizer and config
with open('tokenizer/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
with open('tokenizer/config.pkl', 'rb') as f:
    config = pickle.load(f)
    max_sequence_len = config['max_sequence_len']

# Load custom layers
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_seq_len, d_model, **kwargs):
        super().__init__(**kwargs)
        self.max_seq_len = max_seq_len
        self.d_model = d_model
        self.pos_encoding = self.positional_encoding(max_seq_len, d_model)

    def positional_encoding(self, max_seq_len, d_model):
        position = np.arange(max_seq_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        pos_encoding = np.zeros((max_seq_len, d_model))
        pos_encoding[:, 0::2] = np.sin(position * div_term)
        pos_encoding[:, 1::2] = np.cos(position * div_term)
        return tf.cast(pos_encoding[np.newaxis, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

    def get_config(self):
        config = super().get_config()
        config.update({"max_seq_len": self.max_seq_len, "d_model": self.d_model})
        return config

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.dropout_rate = dropout

        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "dropout": self.dropout_rate
        })
        return config

def generate_text(model, seed_text, next_words=10, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs + 1e-10) / temperature
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))

        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        if output_word:
            seed_text += " " + output_word

    return seed_text

# Seeds and temperatures from assignment
seed_words = ["محبت", "دل", "شام", "یاد", "خوشی"]
temperatures = [0.7, 1.0, 1.3]
optimizers = ['adam', 'rmsprop', 'sgd']

# Generate samples
results = []

for opt in optimizers:
    model = tf.keras.models.load_model(
        f'transformer_{opt}_model.keras',
        custom_objects={'TransformerBlock': TransformerBlock, 'PositionalEncoding': PositionalEncoding}
    )

    print(f"\n{opt.upper()} GENERATIONS:")

    for seed in seed_words:
        for temp in temperatures:
            generated = generate_text(model, seed, next_words=10, temperature=temp)
            print(f"  [{seed}] T={temp}: {generated}")

            results.append({
                'Model': 'Transformer',
                'Optimizer': opt,
                'Seed': seed,
                'Temperature': temp,
                'Generated Text': generated
            })

# Save to CSV
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('transformer_generations.csv', index=False)
print(f"\nSaved {len(results)} generations to transformer_generations.csv")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(



ADAM GENERATIONS:
  [محبت] T=0.7: محبت میں ہم نہ مجھے کہ جیسے غم کے یاد ہے
  [محبت] T=1.0: محبت ساقی کو اے داغؔ البتہ لوگ اپنی ڈھونڈھتے ہے چھو
  [محبت] T=1.3: محبت سے بلاتے کی صبح ریشۂ سے ارادوں کہ ملتا ہے
  [دل] T=0.7: دل کدے سے کیا ہے کہ جو نہ ہوا کب وصال
  [دل] T=1.0: دل کا سونپ نہ کیوں تھا نکالے ہے کیا ہو ہے
  [دل] T=1.3: دل خوں حقیقت پینا اگر جو عیش میں ہے نہ گئی
  [شام] T=0.7: شام ہے ان کا بھی ہے بات ہیں ہم نہ آیا
  [شام] T=1.0: شام پہ مجھ سے آگ کوشی پھونکے کے زباں بھی کر
  [شام] T=1.3: شام گا عہد بشر و وفا کیوں کنج محبت یار خوں
  [یاد] T=0.7: یاد ہے کہ دوا نہاں میں بھی نہ ہے ترے ساغر
  [یاد] T=1.0: یاد سادہ عرفاں جاؤ تھی تجھ کو نظر ہے مقتل سے
  [یاد] T=1.3: یاد چھوڑ جدا مار ہوا یوں رہی ہے ہوتا ہوں مگر
  [خوشی] T=0.7: خوشی کے عالم بھی کہ یہ استعارہ ہیں بھی ہے نہ
  [خوشی] T=1.0: خوشی ہیں بلبل کے دل سے ولے جو یوں تہمتیں تھا
  [خوشی] T=1.3: خوشی گیا ملتا ہیں مصور کبھی تمہارا افزائی آدمی گزر نہیں


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(



RMSPROP GENERATIONS:
  [محبت] T=0.7: محبت نظر آئی ہی نہیں ہے کہ میں تو ہے یہ
  [محبت] T=1.0: محبت کی شکیبا بھی ریشم کا ہائے چنگ و نہ کر
  [محبت] T=1.3: محبت نظم مرغولہ پیوند خطائیں مردانے جام بیعت ہو جائے گا
  [دل] T=0.7: دل میں درد و بے قدر و ہر ایک پر بہ
  [دل] T=1.0: دل میں اب روشنی سے فوٹو رہی کا مگر اب ہو
  [دل] T=1.3: دل محسوس حویلیوں بان کیا غالبؔ اور وہ جناب معصیت گل
  [شام] T=0.7: شام کی کار ہے اور جب بھی نہیں ملتا جاتی جاتی
  [شام] T=1.0: شام غم عشق کو قافلہ عارض توقع افسون کے کے حسابوں
  [شام] T=1.3: شام بدل درمیاں ہنسائیں جو لکھوائے دی فکر کھیلتا لیتے نصیرؔ
  [یاد] T=0.7: یاد رہ گیا ہے دل میں حسن کو کیا چیز ہے
  [یاد] T=1.0: یاد مگر طرح سے تجلیات کچھ نہیں جاتا کا؟ لیے تھا
  [یاد] T=1.3: یاد من کشاں ضد بندوں کسان نامرد امانتیں سیکھ پیار ہوتا
  [خوشی] T=0.7: خوشی سے تو اس کی یاد آئی تری یاد کی ہے
  [خوشی] T=1.0: خوشی کا ہوش شہپر آتا ہے پھر پیالہ بد قامت سے
  [خوشی] T=1.3: خوشی دوسری مستوں سے ملک ہجرتوں بھرتے گرو بھی پلا کر


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(



SGD GENERATIONS:
  [محبت] T=0.7: محبت میں بھی تو نہ جا رہا ہوں میں نہ کر
  [محبت] T=1.0: محبت تب کرنا بانگ دل شکستہ درویش تھا مزا کیا ہوا
  [محبت] T=1.3: محبت ہمارا کتاب فدائے حجاب تحقیق پیا باقی مکر بیٹھا کیا
  [دل] T=0.7: دل سے ہم بھی تو نے بے ہنر بھی نہیں ہوتی
  [دل] T=1.0: دل میں تیری لاگ میں دیوانۂ تو مرے صنم عقیدے کوئی
  [دل] T=1.3: دل آج بھی غصے برپا کوچ لینا ہار رہ سروکار اور
  [شام] T=0.7: شام کے تو نے جو اکثر یہ کیا ہو تو کیا
  [شام] T=1.0: شام بھی چپ سی جھوٹا جانوں کو بیٹھیں گے دو چار
  [شام] T=1.3: شام انتظار بھی تو کہکشانی میرے پاس پایا نہیں جاتا ہے
  [یاد] T=0.7: یاد بھی کیا کیا کیا ہے تیرا اتنا بھی نہ آئے
  [یاد] T=1.0: یاد کو چہرہ کی چاپ اترے ادب کے ایامے نکلا تھا
  [یاد] T=1.3: یاد طول پر میرے لیے جلد تری محفل نشہ تماشا وحشت
  [خوشی] T=0.7: خوشی نے نہ رہے گا کوئی اور واقف بازی ہے تو
  [خوشی] T=1.0: خوشی کی زباں ہے کی امید و حجاب بہت ہے اسدؔ
  [خوشی] T=1.3: خوشی دم اشک منحرف تو دینا تھا دل جل نہیں کیا

Saved 45 generations to transformer_generations.csv


In [2]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Load tokenizer and config
with open('tokenizer/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
with open('tokenizer/config.pkl', 'rb') as f:
    config = pickle.load(f)
    max_sequence_len = config['max_sequence_len']

def generate_text(model, seed_text, next_words=10, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs + 1e-10) / temperature
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))

        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        if output_word:
            seed_text += " " + output_word

    return seed_text

seed_words = ["محبت", "دل", "شام", "یاد", "خوشی"]
temperatures = [0.7, 1.0, 1.3]
optimizers = ['adam', 'rmsprop', 'sgd']

results = []

for opt in optimizers:
    model = tf.keras.models.load_model(f'rnn_{opt}_model.keras')

    print(f"\n{opt.upper()} GENERATIONS:")

    for seed in seed_words:
        for temp in temperatures:
            generated = generate_text(model, seed, next_words=10, temperature=temp)
            print(f"  [{seed}] T={temp}: {generated}")

            results.append({
                'Model': 'RNN',
                'Optimizer': opt,
                'Seed': seed,
                'Temperature': temp,
                'Generated Text': generated
            })

df = pd.DataFrame(results)
df.to_csv('rnn_generations.csv', index=False)
print(f"\nSaved {len(results)} generations to rnn_generations.csv")


ADAM GENERATIONS:
  [محبت] T=0.7: محبت میں بھی اس سے کیا ہے کہ دیکھو شاید لگتا
  [محبت] T=1.0: محبت تو مدت ہے پیا پہ کہہ ہوگا اب ہی کیا
  [محبت] T=1.3: محبت غم منزل ملکوتی پا رس میں الرسل وہاں سنا رہا
  [دل] T=0.7: دل کو نہ تو بھی کیا میرے آگے بھی ہے اور
  [دل] T=1.0: دل آوارہ بولے تو وہی خوؤں ہی کر جائے گا پاس
  [دل] T=1.3: دل آغوشی کاروبار کے کرب کا طرف کرنا ہے جل جاؤں
  [شام] T=0.7: شام نظر کو عالم ہو کوئی ہم میں بھی ہو گا
  [شام] T=1.0: شام پہ پائی جھکوں یہاں نظر نشے کا آج کیوں غالبؔ
  [شام] T=1.3: شام خیز باز تو آہیں لانا کہاں بھیجا دے اسی کے
  [یاد] T=0.7: یاد دوں کئی شہر میں نہ ہوتا ہے جہاں میں ہے
  [یاد] T=1.0: یاد کہاں اپنی کے عیار نہیں رہا ہے دور طرب کی
  [یاد] T=1.3: یاد اپناتا مرہم گیاہ لہلہاتے پسند مانگ ہو گئیں مرا عمر
  [خوشی] T=0.7: خوشی سے تو ہے عشق یہ ہو جائے گا کیا تو
  [خوشی] T=1.0: خوشی بھی ہیں سبھی دو لیے بجھنے لگے نکلے ہے کیا
  [خوشی] T=1.3: خوشی شکل تو لاکھ جب ڈبو زنخداں کی گھڑی پریشان متکبر

RMSPROP GENERATIONS:
  [محبت] T=0.7: محبت میں جو کہ سو کچھ ہے میں استق

In [3]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

with open('tokenizer/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
with open('tokenizer/config.pkl', 'rb') as f:
    config = pickle.load(f)
    max_sequence_len = config['max_sequence_len']

def generate_text(model, seed_text, next_words=10, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs + 1e-10) / temperature
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))

        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        if output_word:
            seed_text += " " + output_word

    return seed_text

seed_words = ["محبت", "دل", "شام", "یاد", "خوشی"]
temperatures = [0.7, 1.0, 1.3]
optimizers = ['adam', 'rmsprop', 'sgd']

results = []

for opt in optimizers:
    model = tf.keras.models.load_model(f'lstm_{opt}_model.keras')

    print(f"\n{opt.upper()} GENERATIONS:")

    for seed in seed_words:
        for temp in temperatures:
            generated = generate_text(model, seed, next_words=10, temperature=temp)
            print(f"  [{seed}] T={temp}: {generated}")

            results.append({
                'Model': 'LSTM',
                'Optimizer': opt,
                'Seed': seed,
                'Temperature': temp,
                'Generated Text': generated
            })

df = pd.DataFrame(results)
df.to_csv('lstm_generations.csv', index=False)
print(f"\nSaved {len(results)} generations to lstm_generations.csv")


ADAM GENERATIONS:
  [محبت] T=0.7: محبت میں بر میں نہیں گلا بھی تو کیا تھا ہم
  [محبت] T=1.0: محبت دل تلک تھیں وو قدموں کی مدد بڑھانا کچھ ہو
  [محبت] T=1.3: محبت کامی و صد نزدیک لیٹے نشاں بھی سکتا تھا پا
  [دل] T=0.7: دل سے کوئی آنکھ دیں ہر نام میں تو نہیں ہوگا
  [دل] T=1.0: دل دم لگی روشن بھی کرنے کی اس نے سایا نہیں
  [دل] T=1.3: دل نے کاش کیا چاند زلف ہی ہے ایسے روانی کشا
  [شام] T=0.7: شام کی واسطے مدھم ہے کبھی تا ایک جان کر جائے
  [شام] T=1.0: شام مرے جویائے و بندۂ مافات تھا کہ نہ رہا گے
  [شام] T=1.3: شام چند سر‌ ہیں طعام خاک دوست نہیں ہے نے بوچھار
  [یاد] T=0.7: یاد کی مجنوں نہیں کبھی غم و بہتے کہ ہے اس
  [یاد] T=1.0: یاد تھے تجھ شوخ خنداں کے سخت کیوں نہ گا ہے
  [یاد] T=1.3: یاد آ کا تلخئ تیر ہوگا دشمن زار کا چاک کس
  [خوشی] T=0.7: خوشی کے کشتی سے ترے شوق میں بھی ملا ہے کہ
  [خوشی] T=1.0: خوشی کب میں تیری عیبوں ہی کر ہے کیا وہی ہی
  [خوشی] T=1.3: خوشی خواب ناکامی ناسازیٔ ہنسائی ہو مجھے آتا ہے سبھی سعیٔ

RMSPROP GENERATIONS:
  [محبت] T=0.7: محبت کی ہی تر نہیں ہے تری پر گردش کے سات

In [2]:
import pandas as pd
import numpy as np

# Load all generation CSVs
rnn_df = pd.read_csv('rnn_generations.csv')
lstm_df = pd.read_csv('lstm_generations.csv')
transformer_df = pd.read_csv('transformer_generations.csv')

all_gens = pd.concat([rnn_df, lstm_df, transformer_df], ignore_index=True)

# Calculate metrics for each generation
results = []

for idx, row in all_gens.iterrows():
    text = row['Generated Text']
    words = text.split()

    # Vocabulary Diversity
    unique_words = len(set(words))
    total_words = len(words)
    vocab_diversity = unique_words / total_words if total_words > 0 else 0

    # Repetition Rate (count repeated bigrams)
    bigrams = [' '.join(words[i:i+2]) for i in range(len(words)-1)]
    repeated_bigrams = len(bigrams) - len(set(bigrams))

    # Average Word Length
    avg_word_length = np.mean([len(word) for word in words]) if words else 0

    results.append({
        'Model': row['Model'],
        'Optimizer': row['Optimizer'],
        'Seed': row['Seed'],
        'Temperature': row['Temperature'],
        'Vocab Diversity': vocab_diversity,
        'Repetition Rate': repeated_bigrams,
        'Avg Word Length': avg_word_length
    })

metrics_df = pd.DataFrame(results)

# Aggregate by Model-Optimizer combination
summary = metrics_df.groupby(['Model', 'Optimizer']).agg({
    'Vocab Diversity': 'mean',
    'Repetition Rate': 'mean',
    'Avg Word Length': 'mean'
}).reset_index()

print("\nQUANTITATIVE METRICS - BY MODEL-OPTIMIZER")
print(summary.to_string(index=False))

# Save detailed and summary metrics
metrics_df.to_csv('generation_metrics_detailed.csv', index=False)
summary.to_csv('generation_metrics_summary.csv', index=False)
print("\nSaved: generation_metrics_detailed.csv, generation_metrics_summary.csv")


QUANTITATIVE METRICS - BY MODEL-OPTIMIZER
      Model Optimizer  Vocab Diversity  Repetition Rate  Avg Word Length
       LSTM      adam         0.981818         0.000000         3.090909
       LSTM   rmsprop         0.969697         0.000000         3.200000
       LSTM       sgd         0.939394         0.000000         3.018182
        RNN      adam         0.969697         0.000000         3.133333
        RNN   rmsprop         0.939394         0.000000         3.333333
        RNN       sgd         0.939394         0.000000         3.121212
Transformer      adam         0.969697         0.000000         3.036364
Transformer   rmsprop         0.951515         0.000000         3.230303
Transformer       sgd         0.933333         0.066667         3.157576

Saved: generation_metrics_detailed.csv, generation_metrics_summary.csv
